# Top Docs for Wikipedia 50-topics models

In [58]:
import sys
import os
import pathlib
import numpy as np
import json
import pandas as pd
from scipy import sparse

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

if 'src.topics_docs_selection.doc_selector' in sys.modules:
    print("doc_selector already in modules")
    del sys.modules['src.topics_docs_selection.doc_selector']

from src.topics_docs_selection.doc_selector import DocSelector

doc_selector already in modules


In [59]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [60]:
def load_model_info(model_path):
    if model_path.stem == "mallet":
        thetas = np.load(model_path / "doctopics.npz.npy")
        betas = np.load(model_path / "beta.npy")
    elif model_path.stem == "ctm":
        thetas = np.load(model_path / "train.theta.npy")
        betas = np.load(model_path / "beta.npy")
    else:
        thetas = sparse.load_npz(pathlib.Path(model_path) / "thetas.npz").toarray()
        betas = np.load(model_path / "betas.npy")
    
    try:
        with open(model_path / "vocab.json") as infile:
            vocab_w2id = json.load(infile)
    except:
        vocab_w2id = {}
        with (pathlib.Path(model_path)/'vocab.txt').open('r', encoding='utf8') as fin:
            for i, line in enumerate(fin):
                wd = line.strip()
                vocab_w2id[wd] = i
    
    try:
        bow = np.load(model_path / "bow_mat.npy")
    except:
        bow = None
    
    vocab_id2w = dict(zip(vocab_w2id.values(), vocab_w2id.keys()))
    keys = [
        [vocab_id2w[idx]
            for idx in row.argsort()[::-1][:15]]
        for row in betas
    ]
        
    return thetas, betas, vocab_w2id, bow, keys

def create_top_docs_dataframe(method, top_docs, df, keys, thetas):
    return pd.DataFrame({
        "id_tpc": range(len(thetas.T)),
        "keys": keys,
        **{f"top doc {i+1}": [df.text.iloc[top_docs[method][k][i]][0:1000] for k in range(len(thetas.T))] for i in range(5)}
    })

def create_top_docs_thetas_dataframe(method, top_docs, df, keys, thetas):
    return pd.DataFrame({
        "id_tpc": range(len(thetas.T)),
        "keys": keys,
        **{f"top doc {i+1}": [thetas.T[k][top_docs[method][k][i]] for k in range(len(thetas.T))] for i in range(5)}
    })

def reduce_threshold(matrix, initial_threshold, decrement=0.01):
    threshold = initial_threshold
    while True:
        modified_matrix = matrix.copy()
        satisfied = True

        for col in range(matrix.shape[1]):
            if np.all(matrix[:, col] <= threshold):
                satisfied = False
                break
        
        if satisfied:
            break
        
        threshold -= decrement

    return threshold

In [61]:
mallet_path = pathlib.Path("/export/usuarios_ml4ds/lbartolome/Repos/umd/theta-evaluation/data/models/mallet")
ctm_path = pathlib.Path("/export/usuarios_ml4ds/lbartolome/Repos/umd/theta-evaluation/data/models/ctm")
bert_path = pathlib.Path("/export/usuarios_ml4ds/lbartolome/Repos/umd/theta-evaluation/data/models/trained/bertopic_wiki_50_ctm_emb_pre")
path_save_tops = pathlib.Path("/export/usuarios_ml4ds/lbartolome/Repos/umd/theta-evaluation/data/top_docs")
corpus_path = pathlib.Path("/export/usuarios_ml4ds/lbartolome/Repos/umd/theta-evaluation/data/train.metadata.jsonl")
text_column = "tokenized_text"

In [29]:
# Read documents
df = pd.read_json(corpus_path, lines=True)

df["id_top"] = range(len(df))
df["len"] = df[text_column].apply(lambda x: len(x.split()))
df["text_split"] = df[text_column].apply(lambda x: x.split())
corpus = df["text_split"].values.tolist()
#df.head()

In [62]:
methods = ['thetas', 'thetas_sample', 'thetas_thr', 'elbow']#'sall', 'spart', 's3'
n_top_words_s3 = 100
n_top_docs = 5

doc_selector = DocSelector()

## Mallet

In [31]:
thetas, betas, vocab_w2id, bow, keys = load_model_info(mallet_path)

In [32]:
max_thetas = np.max(thetas)
min_thetas = np.min(thetas)
print(f"Max/min thetas: {max_thetas} / {min_thetas}")
thr = (0.5,0.9)

Max/min thetas: 0.999596118927002 / 9.460147794015938e-07


In [33]:
top_docs = {
    method: doc_selector.get_top_docs(method=method, thetas=thetas, bow=bow, betas=betas, corpus=corpus, vocab_w2id=vocab_w2id,thr=thr, top_words = None, ntop=n_top_docs, model_path=mallet_path)
    for method in methods
}

In [34]:
for method in top_docs.keys():
    print("##" * 50)
    print(f"-- -- METHOD: {method}")
    df_method = create_top_docs_thetas_dataframe(method, top_docs, df, keys, thetas)
    display(df_method.iloc[0:9])
    (path_save_tops / mallet_path.name).mkdir(exist_ok=True)
    this_path_save = path_save_tops / mallet_path.name / f"{method}_top_docs.xlsx"
    df_method.to_excel(this_path_save, index=False)
    print("\n")

####################################################################################################
-- -- METHOD: thetas


,id_tpc,keys,top doc 1,top doc 2,top doc 3,top doc 4,top doc 5
0,0,"[ship, ships, war, german, fleet, guns, tons, british, torpedo, long, class, cruiser, service, italian, mph]",0.998030,0.997481,0.997357,0.997273,0.997177
1,1,"[british, ships, french, ship, fleet, spanish, island, captain, admiral, naval, port, coast, sea, battle, crew]",0.989845,0.986242,0.975379,0.974118,0.972367
2,2,"[horse, horses, white, breed, dog, black, breeds, dogs, breeding, known, animals, century, riding, type, bred]",0.994985,0.982282,0.976747,0.929790,0.926588
3,3,"[match, australia, runs, test, innings, england, cricket, team, wickets, scored, second, matches, took, australian, played]",0.996480,0.995988,0.986190,0.984264,0.982386
4,4,"[station, line, railway, bridge, trains, new, service, construction, built, train, services, stations, opened, tunnel, london]",0.995329,0.992506,0.986704,0.985714,0.983889
5,5,"[species, shark, long, sharks, females, fish, found, birds, small, large, males, range, bird, brown, white]",0.998083,0.997636,0.997168,0.996253,0.995912
6,6,"[police, said, death, prison, murder, case, found, trial, people, killed, later, evidence, arrested, reported, court]",0.824423,0.762204,0.756807,0.754640,0.744599
7,7,"[world, won, games, olympics, time, olympic, event, round, team, final, gold, medal, summer, events, record]",0.995232,0.994828,0.994677,0.994189,0.994031
8,8,"[aircraft, air, flight, wing, squadron, flying, fighter, pilot, mission, engine, ground, production, crew, pilots, flew]",0.968833,0.963443,0.958630,0.952006,0.951871




####################################################################################################
-- -- METHOD: thetas_sample


,id_tpc,keys,top doc 1,top doc 2,top doc 3,top doc 4,top doc 5
0,0,"[ship, ships, war, german, fleet, guns, tons, british, torpedo, long, class, cruiser, service, italian, mph]",0.249549,0.519919,0.739824,0.524576,0.946214
1,1,"[british, ships, french, ship, fleet, spanish, island, captain, admiral, naval, port, coast, sea, battle, crew]",0.235957,0.860813,0.922306,0.261798,0.089635
2,2,"[horse, horses, white, breed, dog, black, breeds, dogs, breeding, known, animals, century, riding, type, bred]",0.763518,0.982282,0.642796,0.891696,0.340117
3,3,"[match, australia, runs, test, innings, england, cricket, team, wickets, scored, second, matches, took, australian, played]",0.307805,0.881032,0.966975,0.954617,0.764739
4,4,"[station, line, railway, bridge, trains, new, service, construction, built, train, services, stations, opened, tunnel, london]",0.930546,0.588063,0.321718,0.789884,0.058182
5,5,"[species, shark, long, sharks, females, fish, found, birds, small, large, males, range, bird, brown, white]",0.994007,0.048944,0.772550,0.583537,0.989631
6,6,"[police, said, death, prison, murder, case, found, trial, people, killed, later, evidence, arrested, reported, court]",0.293760,0.343408,0.052486,0.590214,0.149524
7,7,"[world, won, games, olympics, time, olympic, event, round, team, final, gold, medal, summer, events, record]",0.925251,0.058858,0.145774,0.626608,0.440767
8,8,"[aircraft, air, flight, wing, squadron, flying, fighter, pilot, mission, engine, ground, production, crew, pilots, flew]",0.861529,0.741067,0.798402,0.317632,0.101908




####################################################################################################
-- -- METHOD: thetas_thr


,id_tpc,keys,top doc 1,top doc 2,top doc 3,top doc 4,top doc 5
0,0,"[ship, ships, war, german, fleet, guns, tons, british, torpedo, long, class, cruiser, service, italian, mph]",0.899761,0.896811,0.894165,0.893444,0.892741
1,1,"[british, ships, french, ship, fleet, spanish, island, captain, admiral, naval, port, coast, sea, battle, crew]",0.893820,0.888176,0.879852,0.877769,0.875201
2,2,"[horse, horses, white, breed, dog, black, breeds, dogs, breeding, known, animals, century, riding, type, bred]",0.897272,0.897051,0.891696,0.891580,0.891322
3,3,"[match, australia, runs, test, innings, england, cricket, team, wickets, scored, second, matches, took, australian, played]",0.899576,0.893354,0.892173,0.891980,0.889254
4,4,"[station, line, railway, bridge, trains, new, service, construction, built, train, services, stations, opened, tunnel, london]",0.897884,0.897677,0.896002,0.894659,0.894405
5,5,"[species, shark, long, sharks, females, fish, found, birds, small, large, males, range, bird, brown, white]",0.899338,0.898731,0.896419,0.896383,0.889896
6,6,"[police, said, death, prison, murder, case, found, trial, people, killed, later, evidence, arrested, reported, court]",0.824423,0.762204,0.756807,0.754640,0.744599
7,7,"[world, won, games, olympics, time, olympic, event, round, team, final, gold, medal, summer, events, record]",0.898961,0.898423,0.898331,0.897278,0.895956
8,8,"[aircraft, air, flight, wing, squadron, flying, fighter, pilot, mission, engine, ground, production, crew, pilots, flew]",0.898050,0.893859,0.893809,0.890589,0.887420




####################################################################################################
-- -- METHOD: elbow


,id_tpc,keys,top doc 1,top doc 2,top doc 3,top doc 4,top doc 5
0,0,"[ship, ships, war, german, fleet, guns, tons, british, torpedo, long, class, cruiser, service, italian, mph]",0.981978,0.936753,0.944621,0.839323,0.988495
1,1,"[british, ships, french, ship, fleet, spanish, island, captain, admiral, naval, port, coast, sea, battle, crew]",0.850674,0.825192,0.967896,0.888176,0.875201
2,2,"[horse, horses, white, breed, dog, black, breeds, dogs, breeding, known, animals, century, riding, type, bred]",0.874607,0.816356,0.705635,0.994985,0.861737
3,3,"[match, australia, runs, test, innings, england, cricket, team, wickets, scored, second, matches, took, australian, played]",0.967364,0.948624,0.954617,0.977677,0.892173
4,4,"[station, line, railway, bridge, trains, new, service, construction, built, train, services, stations, opened, tunnel, london]",0.986704,0.937873,0.981973,0.913557,0.830021
5,5,"[species, shark, long, sharks, females, fish, found, birds, small, large, males, range, bird, brown, white]",0.938952,0.973211,0.813735,0.871476,0.997636
6,6,"[police, said, death, prison, murder, case, found, trial, people, killed, later, evidence, arrested, reported, court]",0.566526,0.642661,0.663064,0.570501,0.550310
7,7,"[world, won, games, olympics, time, olympic, event, round, team, final, gold, medal, summer, events, record]",0.912944,0.993889,0.981664,0.893868,0.986899
8,8,"[aircraft, air, flight, wing, squadron, flying, fighter, pilot, mission, engine, ground, production, crew, pilots, flew]",0.949666,0.867591,0.951871,0.721232,0.741067


In [35]:
for method in top_docs.keys():
    print("##" * 50)
    print(f"-- -- METHOD: {method}")
    df_method = create_top_docs_dataframe(method, top_docs, df, keys, thetas)
    display(df_method.iloc[0:9])
    (path_save_tops / mallet_path.name).mkdir(exist_ok=True)
    this_path_save = path_save_tops / mallet_path.name / f"{method}_top_docs.xlsx"
    df_method.to_excel(this_path_save, index=False)
    print("\n")

####################################################################################################
-- -- METHOD: thetas


,id_tpc,keys,top doc 1,top doc 2,top doc 3,top doc 4,top doc 5
0,0,"[ship, ships, war, german, fleet, guns, tons, british, torpedo, long, class, cruiser, service, italian, mph]","Bremen-class cruiser = The Bremen class was a group of seven light cruisers built for the Imperial German Navy in the early 1900s . The seven ships , Bremen , Hamburg , Berlin , Lübeck , München , Leipzig , and Danzig , were an improvement upon the previous Gazelle class . They were significantly larger than the earlier class , and were faster and better armored . Like the Gazelles , they were armed with a main battery of ten 10.5 cm SK L / 40 guns and a pair of torpedo tubes . The ships of the Bremen class served in a variety of roles , from overseas cruiser to fleet scout to training ship . Bremen and Leipzig were deployed to the American and Asian stations , respectively , while the other five ships remained in German waters with the High Seas Fleet . At the outbreak of World War I in August 1914 , Leipzig was in the Pacific Ocean in the East Asia Squadron ; she saw action at the Battle of Coronel in November and was sunk a month later at the Battle of the Falkland Islands . Bremen","Italian battleship Emanuele Filiberto = The Emanuele Filiberto was a pre-dreadnought battleship built for the Italian Navy ( Italian : Regia Marina ) during the 1890s . Her keel was laid down in October 1893 and she was launched in September 1897 ; work was completed in April 1902 . She had one sister ship , Ammiraglio di Saint Bon , the lead ship of the Ammiraglio di Saint Bon class . She was armed with a main battery of four 10-inch ( 254 mm ) guns and was capable of a speed in excess of 18 knots ( 33 km / h ; 21 mph ) . Emanuele Filiberto served in the active squadron of the Italian navy for the first several years of her career . She was assigned to the 3rd Division during the Italo-Turkish War of 1911 – 1912 . During the war , she was involved in the assaults on Tripoli in North Africa and on the island of Rhodes in the eastern Mediterranean Sea . She was obsolescent by World War I and was slated to be broken up in 1914 – 15 , but the need for warships granted Emanuele Filiberto a","SMS Graudenz = SMS Graudenz was the lead ship of her class of light cruisers . She had one sister ship , SMS Regensburg . The ship was built by the German Kaiserliche Marine ( Imperial Navy ) in the Kaiserliche Werft shipyard in Kiel , laid down in 1912 and commissioned into the High Seas Fleet in August 1914 , days after the outbreak of World War I. She was named for the then-German town of Graudenz ( now Grudziądz , Poland ) . The ship was armed with a main battery of twelve 10.5 cm SK L / 45 guns and had a top speed of 27.5 knots ( 50.9 km / h ; 31.6 mph ) . Graudenz saw extensive service during World War I , including serving as part of the reconnaissance screen for the battlecruisers of the I Scouting Group during the raid on Scarborough , Hartlepool and Whitby in December 1914 . The ship also took part in the Battle of Dogger Bank in January 1915 , and the Battle of the Gulf of Riga in August 1915 . She had been damaged by a mine and was unable to participate in the Battle of Jut","Graudenz-class cruiser = The Graudenz class of light cruisers was a class of two ships built for the German Imperial Navy . The class comprised SMS Graudenz and SMS Regensburg . The ships both were laid down in 1912 , launched in October 1913 and April 1914 and commissioned in August 1914 and January 1915 , respectively . They were armed with a main battery of twelve 10.5 cm guns , though over the course of their careers , they were rearmed with seven more powerful 15 cm guns . They displaced 6,382 t ( 6,281 long tons ) at full load and were rated at a top speed of 27.5 knots ( 50.9 km / h ; 31.6 mph ) Both ships saw extensive service during World War I , primarily in the reconnaissance forces of the High Seas Fleet . They participated in several raids on the British coast , screening for the battlecru



####################################################################################################
-- -- METHOD: thetas_sample


,id_tpc,keys,top doc 1,top doc 2,top doc 3,top doc 4,top doc 5
0,0,"[ship, ships, war, german, fleet, guns, tons, british, torpedo, long, class, cruiser, service, italian, mph]","German destroyer Z12 Erich Giese = Z12 Erich Giese was a Type 1934A-class destroyer built for Nazi Germany 's Kriegsmarine in the late 1930s . At the beginning of World War II , the ship was used in the German Bight to lay minefields in German waters . In late 1939 the ship made one successful minelaying sortie off the English coast that claimed two merchant ships . While returning from that sortie , she torpedoed a British destroyer without being detected and continued on her way . During the early stages of the Norwegian Campaign , Erich Giese fought in both naval Battles of Narvik in mid-April 1940 and was sunk by British destroyers during the Second Battle of Narvik . = = Design and description = = Erich Giese had an overall length of 119 meters ( 390 ft 5 in ) and was 114 meters ( 374 ft 0 in ) long at the waterline . The ship had a beam of 11.30 meters ( 37 ft 1 in ) , and a maximum draft of 4.23 meters ( 13 ft 11 in ) . She displaced 2,171 metric tons ( 2,137 long tons ) at stan","Scharnhorst-class cruiser = The Scharnhorst class was the last traditional class of armored cruisers built by the Kaiserliche Marine . The class comprised two ships , Scharnhorst and Gneisenau . They were larger than the Roon-class cruisers that preceded them ; the extra size was used primarily to increase the main armament of 21 cm ( 8.2 inch ) guns from four to eight . The ships were the first German cruiser to reach equality with their British counterparts . The ships were named after 19th century Prussian army reformers , Gerhard von Scharnhorst and August von Gneisenau . Built for overseas service , Scharnhorst and Gneisenau were assigned to the East Asia Squadron in 1909 and 1910 , respectively . Scharnhorst relieved the old armored cruiser Fürst Bismarck as the squadron flagship , which had been on station since 1900 . Both ships had short careers ; shortly before the outbreak of World War I , the ships departed the German colony at Tsingtao . On 1 November 1914 , the ships dest","Ottoman ironclad Âsâr-ı Tevfik = Âsâr-ı Tevfik ( Ottoman Turkish : God 's Favor ) was an ironclad warship of the Ottoman Navy built in the 1860s , the only member of her class . She was built as part of a major expansion program for the Ottoman fleet in the 1860s following the Crimean War . Âsâr-ı Tevfik was a 4,600-metric-ton ( 4,500-long-ton ; 5,100-short-ton ) barbette ship armed with a main battery of eight 220-millimeter ( 8.7 in ) guns in a central battery . In 1903 – 1906 , the ship was extensively rebuilt in Germany and a new battery of 150 mm ( 5.9 in ) and 120 mm ( 4.7 in ) quick-firing guns replaced the older weapons . Âsâr-ı Tevfik served in the Ottoman fleet for more than four decades . During this period , she saw action in two major wars , the Russo-Turkish War of 1877 – 88 and the First Balkan War in 1913 . During the first conflict , she was torpedoed by a Russian torpedo boat but was only slightly damaged . She took part in the abortive Battle of Elli against the Gree","German destroyer Z15 Erich Steinbrinck = Z15 Erich Steinbrinck was a Type 1934A-class destroyer built for Nazi Germany 's Kriegsmarine in the mid-1930s . The ship was named after the First World War German naval officer Erich Steinbrinck . At the beginning of World War II on 1 September 1939 , the ship was initially deployed to blockade the Polish coast , but she was quickly transferred to the North Sea to lay defensive minefields . In late 1939 and 1940 the ship laid multiple offensive minefields off the English coast that claimed 24 merchant ships and a destroyer . Steinbrinck was under repair for most of the Norwegian Campaign of early 1940 and was transferred to France later that year . After a lengthy refit in Germany , she returned to France in early 1941 where she escorted returning warships , 



####################################################################################################
-- -- METHOD: thetas_thr


,id_tpc,keys,top doc 1,top doc 2,top doc 3,top doc 4,top doc 5
0,0,"[ship, ships, war, german, fleet, guns, tons, british, torpedo, long, class, cruiser, service, italian, mph]","Italian ironclad Sardegna = Sardegna was the third of three Re Umberto-class ironclad battleships built for the Italian Regia Marina ( Royal Navy ) . The ship , named for the island of Sardinia , was laid down in La Spezia in October 1885 , launched in September 1890 , and completed in February 1895 . She was armed with a main battery of four 13.5-inch ( 340 mm ) guns and had a top speed of 20.3 knots ( 37.6 km / h ; 23.4 mph ) — albeit at the cost of armor protection – and she was one of the first warships to be equipped with a wireless telegraph . Sardegna spent the first decade of her career in the Active Squadron of the Italian fleet . Thereafter , she was transferred to the Reserve Squadron , and by 1911 , she was part of the Training Division . She took part in the Italo-Turkish War of 1911 – 12 , where she escorted convoys to North Africa and supported Italian forces ashore by bombarding Ottoman troops . During World War I , Sardegna served as the flagship of the naval forces de","Italian battleship Napoli = Napoli was a Regina Elena-class pre-dreadnought battleship built for the Italian Regia Marina ( Royal Navy ) in 1903 – 08 . She was the last member of the four-ship class , which included the lead ship Regina Elena , Vittorio Emanuele , and Roma . Napoli was armed with a main battery of two 12-inch ( 305 mm ) and twelve 8 in ( 203 mm ) guns , and was capable of a top speed of 21 knots ( 39 km / h ; 24 mph ) . Napoli saw action in the Italo-Turkish War in 1911 and 1912 ; she took part in the attack on Derna , Libya , and the amphibious assaults on the islands of Rhodes and the Dodecanese in the Aegean Sea . Napoli remained in service during World War I in 1915 – 18 , but saw no action as a result of the cautious policies of both the Italian and Austro-Hungarian navies . She remained in the Italian inventory until she was stricken from the naval register in August 1926 and was subsequently broken up for scrap . = = Design = = Napoli was 144.6 meters ( 474 ft )","SM UB-3 = SM UB-3 was a German Type UB I submarine or U-boat in the German Imperial Navy ( German : Kaiserliche Marine ) during World War I. She disappeared on her first patrol in May 1915 , and was the first of her class to be lost . UB-3 was ordered in October 1914 and was laid down at the Germaniawerft shipyard in Kiel in November . UB-3 was a little more than 28 metres ( 92 ft ) in length and displaced between 127 and 142 metric tons ( 125 and 140 long tons ) , depending on whether surfaced or submerged . She carried two torpedoes for her two bow torpedo tubes and was also armed with a deck-mounted machine gun . She was launched and commissioned as SM UB-3 in March 1915 . UB-3 was broken into sections and shipped by rail to the Austro-Hungarian port of Pola in April for reassembly . She officially joined the Pola Flotilla on 1 May and departed on her first patrol for temporary duty in Turkey on 23 May , and was never seen again . A postwar German study concluded that UB-3 was likel","Italian cruiser Coatit = Coatit was a torpedo cruiser of the Italian Regia Marina built in the late 1890s . She was the second and final member of the Agordat class . The ship , which was armed with twelve 76 mm ( 3.0 in ) guns and two 450 mm ( 18 in ) torpedo tubes , was too slow and short-ranged to be able to scout effectively for the fleet , so her career was limited . She saw action during the Italo-Turkish War in 1911 – 12 , where she provided gunfire support to Italian troops in North Africa . She also caused a minor diplomatic incident from an attack on retreating Ottoman soldiers in Anatolia . Coatit was part of an international fleet sent to Constantinople when the city appeared to be at risk of falling to the Bulgarian Army during the First Balkan War . In 1919 , she was converted into a min



####################################################################################################
-- -- METHOD: elbow


,id_tpc,keys,top doc 1,top doc 2,top doc 3,top doc 4,top doc 5
0,0,"[ship, ships, war, german, fleet, guns, tons, british, torpedo, long, class, cruiser, service, italian, mph]","SMS Comet = SMS Comet was an aviso of the Imperial German Navy , the second and final member of the Meteor class . She had one sister ship , Meteor . Comet was laid down at the AG Vulcan shipyard in 1890 , launched in January 1892 , and commissioned in April 1893 . Like her sister , Comet was plagued by excessive vibration and poor handling , which limited her active duty career . She ended her career as a mine hulk in Emden . After the German defeat in World War I , she was sold for scrapping in 1921 . = = Design = = Comet was 79.86 meters ( 262.0 ft ) long overall and had a beam of 9.58 m ( 31.4 ft ) and a maximum draft of 3.68 m ( 12.1 ft ) forward . She displaced 1,117 metric tons ( 1,099 long tons ; 1,231 short tons ) at full combat load . Her propulsion system consisted of two vertical 3-cylinder triple expansion engines . Steam for the engines was provided by four coal-fired locomotive boilers . The ship 's propulsion system provided a top speed of 19.5 kn ( 36.1 km / h ; 22.4 m","German submarine U-162 ( 1941 ) = German submarine U-162 was a Type IXC U-boat of Nazi Germany 's Kriegsmarine during World War II . She was ordered on 25 September 1939 and was laid down on 19 April 1940 at Deutsche Schiff- und Maschinenbau AG , at Seebeck Yard in Bremerhaven , Germany , as yard number 701 . She was launched on 1 March 1941 and commissioned under the command of Korvettenkapitän Jürgen Wattenberg on 9 September of that year . During three war patrols , U-162 sank 14 vessels . However , on 3 September , three British destroyers hunted U-162 down and sank her . Of a crew of fifty-one , only two died . The remainder were taken prisoner and sent to camps in the United States , where they were to remain for the rest of the war . = = Design = = German Type IXC submarines were slightly larger than the original Type IXBs . U-162 had a displacement of 1,120 tonnes ( 1,100 long tons ) when at the surface and 1,232 tonnes ( 1,213 long tons ) while submerged . The U-boat had a tot","SMS Bremen = SMS Bremen ( "" His Majesty 's Ship Bremen "" ) was the lead ship of the seven-vessel Bremen class , built by the Imperial German Navy . She was built by the AG Weser shipyard in Bremen , her namesake city . She was laid down in 1902 , launched in July 1903 , and commissioned into the High Seas Fleet in May 1904 . Armed with a main battery of ten 10.5 cm ( 4.1 in ) guns and two 45 cm ( 18 in ) torpedo tubes , Bremen was capable of a top speed of 22 knots ( 41 km / h ; 25 mph ) . Bremen served on the East American Station for the majority of her career , including the ten years before the outbreak of World War I. She returned to Germany in 1914 before the start of the war . At the onset of hostilities , she was attached to the fleet in the Baltic tasked with containing the Russians . In August 1915 , she participated in the Battle of the Gulf of Riga , but did not see significant action during the battle . Four months later , on 17 December , she struck two Russian naval mine","SMS Schlesien = SMS Schlesien was one of the five Deutschland-class pre-dreadnought battleships built for the Kaiserliche Marine between 1904 and 1906 . Named after the German province of Schlesien , she was built at the Schichau-Werke shipyard in Danzig , where she was launched on 28 May 1906 . She was commissioned into the navy on 5 May 1908 . The ships of her class were already outdated by the time they entered service , being inferior in size , armor , firepower and speed to the revolutionary new battleship HMS Dreadnought . After commissioning , Schlesien was assigned to the I Battle Squadron of the High Seas Fleet . She served with the fleet throughout the first two years of World War I ; by this time she had been transferred to the II Battle Squadron alongside her four sister ships . Schlesie

## CTM

In [36]:
thetas, betas, vocab_w2id, bow, keys = load_model_info(ctm_path)

In [37]:
max_thetas = np.max(thetas)
min_thetas = np.min(thetas)
print(f"Max/min thetas: {max_thetas} / {min_thetas}")
thr = (0.5,0.9)

thr_list = list(thr)

thr_list[1] = reduce_threshold(thetas, thr_list[1], 0.05)
print("Updated thr_list:", thr_list)

# Adjust thr[0] to be at least 0.2 less than thr[1]
while thr_list[0] >= thr_list[1] - 0.2:
    thr_list[0] -= 0.01  
    if thr_list[0] <= 0:
        thr_list[0] = 0
        break

thr = tuple(thr_list)
print("Final thr:", thr)
print("Final thr[0]:", thr[0])

Max/min thetas: 0.9954009354114532 / 1.8919544385198605e-05
Updated thr_list: [0.5, 0.29999999999999977]
Final thr: (0.09999999999999969, 0.29999999999999977)
Final thr[0]: 0.09999999999999969


In [38]:
top_docs = {
    method: doc_selector.get_top_docs(method=method, thetas=thetas, bow=bow, betas=betas, corpus=corpus, vocab_w2id=vocab_w2id,thr=thr, top_words = None, ntop=n_top_docs, model_path=mallet_path)
    for method in methods
}

In [39]:
for method in top_docs.keys():
    print("##" * 50)
    print(f"-- -- METHOD: {method}")
    df_method = create_top_docs_thetas_dataframe(method, top_docs, df, keys, thetas)
    display(df_method.iloc[0:9])
    (path_save_tops / mallet_path.name).mkdir(exist_ok=True)
    this_path_save = path_save_tops / mallet_path.name / f"{method}_top_docs.xlsx"
    df_method.to_excel(this_path_save, index=False)
    print("\n")

####################################################################################################
-- -- METHOD: thetas


,id_tpc,keys,top doc 1,top doc 2,top doc 3,top doc 4,top doc 5
0,0,"[credits, composition, allmusic, listing, charts, piano, drums, beats, liner, label, recording, download, ballad, songwriter, vocals]",0.329688,0.326009,0.324521,0.320877,0.315103
1,1,"[film, release, released, million, story, scene, scenes, script, effects, production, opening, plot, filming, original, director]",0.501727,0.498330,0.490960,0.483859,0.473820
2,2,"[career, hit, games, season, league, baseball, major_league_baseball, signed, home, played, manager, professional, minor, earned, hits]",0.737660,0.723641,0.714612,0.708765,0.705220
3,3,"[work, life, wrote, writing, published, book, written, women, works, history, world, writer, woman, early, books]",0.402178,0.376136,0.367494,0.365637,0.362293
4,4,"[tons, beam, deck, consisted, guns, laid, horsepower, armor, hull, boilers, displaced, steam, mounted, armament, battery]",0.421570,0.419186,0.410072,0.399300,0.389479
5,5,"[different, called, given, form, example, point, elements, real, use, possible, formula, sequence, particular, result, simple]",0.885582,0.860844,0.831554,0.822141,0.818491
6,6,"[war, american, later, wrote, years, began, men, time, british, land, united_states, known, general, early, received]",0.448018,0.428517,0.388362,0.384286,0.383984
7,7,"[game, released, games, player, music, version, original, series, release, story, characters, players, main, character, japan]",0.657687,0.651847,0.594150,0.569258,0.553545
8,8,"[tropical, storm, cyclone, area, cyclones, near, level, wind, low, formed, depression, miles, winds, water, west]",0.421496,0.418954,0.406579,0.401785,0.392336




####################################################################################################
-- -- METHOD: thetas_sample


,id_tpc,keys,top doc 1,top doc 2,top doc 3,top doc 4,top doc 5
0,0,"[credits, composition, allmusic, listing, charts, piano, drums, beats, liner, label, recording, download, ballad, songwriter, vocals]",0.014268,0.014280,0.246999,0.295424,0.013625
1,1,"[film, release, released, million, story, scene, scenes, script, effects, production, opening, plot, filming, original, director]",0.006780,0.392907,0.006807,0.162683,0.111775
2,2,"[career, hit, games, season, league, baseball, major_league_baseball, signed, home, played, manager, professional, minor, earned, hits]",0.002069,0.023523,0.018095,0.449397,0.009531
3,3,"[work, life, wrote, writing, published, book, written, women, works, history, world, writer, woman, early, books]",0.022173,0.179930,0.008320,0.019569,0.011348
4,4,"[tons, beam, deck, consisted, guns, laid, horsepower, armor, hull, boilers, displaced, steam, mounted, armament, battery]",0.012224,0.018149,0.017637,0.009456,0.313316
5,5,"[different, called, given, form, example, point, elements, real, use, possible, formula, sequence, particular, result, simple]",0.015668,0.003616,0.399230,0.102851,0.192275
6,6,"[war, american, later, wrote, years, began, men, time, british, land, united_states, known, general, early, received]",0.236281,0.020566,0.088758,0.278322,0.039884
7,7,"[game, released, games, player, music, version, original, series, release, story, characters, players, main, character, japan]",0.039390,0.020664,0.010361,0.010029,0.036263
8,8,"[tropical, storm, cyclone, area, cyclones, near, level, wind, low, formed, depression, miles, winds, water, west]",0.089400,0.007517,0.007606,0.014288,0.047276




####################################################################################################
-- -- METHOD: thetas_thr


,id_tpc,keys,top doc 1,top doc 2,top doc 3,top doc 4,top doc 5
0,0,"[credits, composition, allmusic, listing, charts, piano, drums, beats, liner, label, recording, download, ballad, songwriter, vocals]",0.298118,0.298068,0.297612,0.296216,0.295444
1,1,"[film, release, released, million, story, scene, scenes, script, effects, production, opening, plot, filming, original, director]",0.299881,0.298767,0.297957,0.296576,0.296187
2,2,"[career, hit, games, season, league, baseball, major_league_baseball, signed, home, played, manager, professional, minor, earned, hits]",0.299455,0.296663,0.292269,0.291120,0.290753
3,3,"[work, life, wrote, writing, published, book, written, women, works, history, world, writer, woman, early, books]",0.297983,0.296374,0.295663,0.293647,0.292484
4,4,"[tons, beam, deck, consisted, guns, laid, horsepower, armor, hull, boilers, displaced, steam, mounted, armament, battery]",0.299092,0.298865,0.298346,0.297985,0.297860
5,5,"[different, called, given, form, example, point, elements, real, use, possible, formula, sequence, particular, result, simple]",0.299226,0.298007,0.293263,0.292882,0.289715
6,6,"[war, american, later, wrote, years, began, men, time, british, land, united_states, known, general, early, received]",0.299785,0.298510,0.298420,0.297476,0.295700
7,7,"[game, released, games, player, music, version, original, series, release, story, characters, players, main, character, japan]",0.299660,0.299135,0.297911,0.297547,0.297229
8,8,"[tropical, storm, cyclone, area, cyclones, near, level, wind, low, formed, depression, miles, winds, water, west]",0.299137,0.298311,0.290756,0.290666,0.289036




####################################################################################################
-- -- METHOD: elbow


,id_tpc,keys,top doc 1,top doc 2,top doc 3,top doc 4,top doc 5
0,0,"[credits, composition, allmusic, listing, charts, piano, drums, beats, liner, label, recording, download, ballad, songwriter, vocals]",0.254434,0.238671,0.258202,0.295444,0.228709
1,1,"[film, release, released, million, story, scene, scenes, script, effects, production, opening, plot, filming, original, director]",0.356946,0.427665,0.328924,0.379547,0.434675
2,2,"[career, hit, games, season, league, baseball, major_league_baseball, signed, home, played, manager, professional, minor, earned, hits]",0.638338,0.562809,0.673049,0.559465,0.679484
3,3,"[work, life, wrote, writing, published, book, written, women, works, history, world, writer, woman, early, books]",0.251907,0.264407,0.233317,0.295663,0.243076
4,4,"[tons, beam, deck, consisted, guns, laid, horsepower, armor, hull, boilers, displaced, steam, mounted, armament, battery]",0.288791,0.345299,0.344471,0.281216,0.399300
5,5,"[different, called, given, form, example, point, elements, real, use, possible, formula, sequence, particular, result, simple]",0.571922,0.600800,0.637443,0.577655,0.885582
6,6,"[war, american, later, wrote, years, began, men, time, british, land, united_states, known, general, early, received]",0.259966,0.352568,0.316938,0.298420,0.278322
7,7,"[game, released, games, player, music, version, original, series, release, story, characters, players, main, character, japan]",0.454563,0.452502,0.477271,0.369149,0.376949
8,8,"[tropical, storm, cyclone, area, cyclones, near, level, wind, low, formed, depression, miles, winds, water, west]",0.418954,0.287934,0.401785,0.302923,0.286694


In [40]:
for method in top_docs.keys():
    print("##" * 50)
    print(f"-- -- METHOD: {method}")
    df_method = create_top_docs_dataframe(method, top_docs, df, keys, thetas)
    display(df_method.iloc[0:9])
    (path_save_tops / ctm_path.name).mkdir(exist_ok=True)
    this_path_save = path_save_tops / ctm_path.name / f"{method}_top_docs.xlsx"
    df_method.to_excel(this_path_save, index=False)
    print("\n")

####################################################################################################
-- -- METHOD: thetas


,id_tpc,keys,top doc 1,top doc 2,top doc 3,top doc 4,top doc 5
0,0,"[credits, composition, allmusic, listing, charts, piano, drums, beats, liner, label, recording, download, ballad, songwriter, vocals]","Oh Blue Christmas = Oh Blue Christmas is an EP by the American band A Fine Frenzy , released in November 2009 in the United States by Virgin Records . It was initially released exclusively through Target just two months following the group 's previous studio album Bomb in a Birdcage . The EP contains cover versions of three popular holiday songs ( "" Blue Christmas "" , "" Winter Wonderland "" and "" Christmas Time Is Here "" from A Charlie Brown Christmas ) as well as three original tracks . According to Alison Sudol , A Fine Frenzy 's leader , the collection of songs was recorded in five days at record producer David Bianco 's studio in Los Angeles . The EP , and "" Christmas Time Is Here "" in particular , received positive critical reception overall . In the United States , Oh Blue Christmas peaked at number 87 on the Billboard 200 , number three on Billboard 's Top Holiday Albums chart and number 28 on the Top Rock Albums chart . A Fine Frenzy was touring throughout the United States arou","The Stripped Mixes = The Stripped Mixes ( released on CD for a limited time as The Motown 50 Mixes ) , is a collection of American recording artist and former Jackson 5 member Michael Jackson 's classic songs . The songs featured on the album are mainly from Jackson 's career as a member of the Jackson 5 from the 1960s to the 1980s ; songs credited to Jackson as a solo artist are from his albums during his Motown-era . Other songs included on the album are "" stripped "" mixes of Jackson material , meaning the songs are of a quieter tone and most of the drums have been removed . Following a surge in Jackson 's popularity after his death in June 2009 , it was confirmed on July 7 , that The Stripped Mixes would be released . The album was made available as a digital download on July 7 , 2009 , and as a CD on July 28 . A remixed song , entitled "" I 'll Be There ( Minus Mix ) "" , was released on iTunes to promote the album on June 9 , 2009 , prior to Jackson 's death . The Stripped Mixes was","A Different Kind of Love Song = "" A Different Kind of Love Song "" is a song by American recording artist Cher , taken from her 24th studio album , Living Proof ( 2002 ) . The song was written and produced by Sigurd Rosnes and Johan Aberg , with additional writing done by Michelle Lewis , and was co-produced by Anders Hansson . The dance-pop song alludes to themes of tragedy , heroism and brotherhood , and was released as a double A-side single with "" The Music 's No Good Without You "" in July 2002 through Warner Bros. Records and WEA . "" A Different Kind of Love Song "" received mostly positive reviews from music critics , who deemed it as one of the album 's highlights , although noting Cher 's heavily processed vocals due to the use of auto-tune . The song charted on a few Billboard components , such as Dance / Club Play Songs , where it reached number one , Hot Dance Music / Maxi-Singles Sales , peaking at number two , and on the Adult Contemporary chart . No accompanying music video","Electric ( Robyn song ) = "" Electric "" is a song by Swedish recording artist Robyn from her second studio album My Truth ( 1999 ) . It was released as the album 's lead single on 29 April 1999 by BMG Sweden . Robyn wrote the track in collaboration with its producers Ulf Lindström and Johan Ekhé . The single artwork features the singer wearing a feather headpiece designed by Sebastian Wahl . Musically , "" Electric "" is an electronic funk song , and the lyrics address unexpected life events that make one feel alive . "" Electric "" was hailed by music critics as a highlight on My Truth . It achieved commercial success in Sweden , with a peak position of number six on the Sverigetopplistan chart and a gold certification by the Swedish Recording Industry Asso



####################################################################################################
-- -- METHOD: thetas_sample


,id_tpc,keys,top doc 1,top doc 2,top doc 3,top doc 4,top doc 5
0,0,"[credits, composition, allmusic, listing, charts, piano, drums, beats, liner, label, recording, download, ballad, songwriter, vocals]","Tomoyo After : It 's a Wonderful Life = Tomoyo After : It 's a Wonderful Life ( 智代アフター ～ It 's a Wonderful Life ～ , Tomoyo Afutā ~ It 's a Wonderful Life ~ ) is a Japanese adult visual novel developed by Key released on November 25 , 2005 for Windows PCs . The game is a spin-off of Key 's earlier all ages game Clannad . Key later released versions of Tomoyo After without the erotic content , and the game was ported to the PlayStation 2 , PlayStation Portable , Xbox 360 and PlayStation 3 under the title Tomoyo After : It 's a Wonderful Life CS Edition ; CS stands for "" consumer software "" . The story follows the lives of Tomoya Okazaki , a young man who recently graduated from high school , and his close friend Tomoyo Sakagami as they start to see more of each other in a romantic relationship . The gameplay in Tomoyo After follows a branching plot line which offers pre-determined scenarios with courses of interaction , and focuses on the appeal of the title character Tomoyo . Being a sp","Basarab I of Wallachia = Basarab I ( Romanian pronunciation : [ basaˈrab ] ) , also known as Basarab the Founder ( Romanian : Basarab Întemeietorul ) , was a voivode , and later the first independent ruler of Wallachia who lived in the first half of the 14th century . Many details of his life are uncertain . Although his name is of Turkic origin , 14th-century sources unanimously state that he was a Vlach . Basarab came into power before 1324 , but the circumstances of his ascension are unknown . According to two popular theories , he succeeded either his father , Thocomerius , or the legendary founder of Wallachia , Radu Negru . A royal charter issued on 26 July 1324 is the first document to reference Basarab . According to the charter , he was subject to Charles I of Hungary as the voivode of Wallachia . Basarab became "" disloyal to the Holy Crown of Hungary "" in 1325 . He seized the Banate of Severin and raided the southern regions of the Kingdom of Hungary . Basarab supported Micha","Long Way to Go ( Gwen Stefani and André 3000 song ) = "" Long Way to Go "" is a song by American singer-songwriter Gwen Stefani with American rapper André 3000 . The song appears as the closing track on Stefani 's debut studio album , Love . Angel . Music . Baby . ( 2004 ) . It was released on November 23 , 2004 , along with the rest of Love . Angel . Music . Baby. by Interscope Records . The track was written by both Stefani and 3000 , while 3000 was the sole producer of the track . Although the song was scrapped from 3000 's critically acclaimed OutKast studio album , The Love Below ( 2003 ) , Stefani and 3000 finalized a reworked version of the song to be included on Stefani 's album . The song prominently features a sample from Martin Luther King , Jr . ' s "" I Have a Dream "" speech in its closing outro . King is not credited for contributed lyrics to the song . Musically , "" Long Way to Go "" is influenced by electronic music and alternative hip hop , with partial influence from both","Strange Overtones = "" Strange Overtones "" is a song recorded by David Byrne and Brian Eno , written by the duo with Leo Abrahams . It was released on August 4 , 2008 by means of free download as the lead single from Byrne 's and Eno 's second collaborative studio album Everything That Happens Will Happen Today ( 2008 ) . "" Strange Overtones "" is an uptempo electronic gospel song , and its lyrics explore the themes of humanity overcoming technology that are central to the album . "" Strange Overtones "" was well received by critics , and was downloaded 40,000 times in its first three days of release . = = Recording and release = = While discussing the 2006 remix of My Life in the Bush of Ghosts at a dinner party , Eno suggested finishing some songs that he had wri



####################################################################################################
-- -- METHOD: thetas_thr


,id_tpc,keys,top doc 1,top doc 2,top doc 3,top doc 4,top doc 5
0,0,"[credits, composition, allmusic, listing, charts, piano, drums, beats, liner, label, recording, download, ballad, songwriter, vocals]","I 'm That Chick = "" I 'm That Chick "" is a song by American singer and songwriter Mariah Carey from her eleventh studio album , E = MC ² ( 2008 ) . A disco track with influences of R & B , it was written by Carey , Johntá Austin and production duo Stargate ( Mikkel S. Eriksen and Tor E. Hermansen ) . It contains elements of a song called "" Off the Wall "" originally performed by Michael Jackson . Rod Temperton , the songs composer , received a songwriting for "" I 'm That Chick "" as result . The track was positively received amongst most music critics , many of whom called it the best song on the album and praised is disco composition . Carey performed "" I 'm That Chick "" on The Hills season premiere party , the 2008 Fashion Rocks , Canadian Idol , and included it on the set-list of her eighth concert tour , The Elusive Chanteuse Show ( 2014 ) . It peaked at number 82 on the U.S. Billboard Hot R & B / Hip-Hop Songs chart . = = Recording and composition = = "" I 'm That Chick "" was written","Sing Like Me = "" Sing Like Me "" is a song by American recording artist Chris Brown . It was released as a promotional single from his third studio album , Graffiti , on November 24 , 2009 in the United States . The number was written by Brown , Big Makk , Keith Thomas , Lorenza "" Big Lo "" Lennon and Atozzio Towns , and produced by Makk , Thomas and Lennon . A pop and R & B ballad , the song contains lyrics about Brown leaving a nightclub with several women . "" Sing Like Me "" received mixed response from critics , and charted for two weeks on the US Hot R & B / Hip-Hop Songs in 2010 , peaking at number eighty-four . = = Background and composition = = "" Sing Like Me "" was written by Chris Brown , Big Makk , Keith Thomas , andf Lorenza "" Big Lo "" Lennon , with Makk , Thomas and Lennon producing the track . It was recorded at The Compound — a recording studio in Orlando , Florida — and Tony Maserati mixed the tune at The Record Plant — a studio in Los Angeles , California . The cover art o","Hot Tottie = "" Hot Tottie "" is a song by recording artist Usher . It was written by Usher , Ester Dean , Jay-Z and Polow da Don , with the latter producing it . The song features guest vocals from rapper Jay-Z and background vocals by Ester Dean . It is the second single in the United States and Canada from his EP , Versus , which is an extension of his sixth studio album , Raymond v. Raymond . The song was sent to rhythmic and urban airplay on August 9 , 2010 . "" Hot Tottie "" samples Big Tymers 's "" Big Ballin ' "" off their 1998 album How You Luv That Vol . 2 . "" Hot Tottie "" is an R & B song with hip hop tones , accompanied by strobing , electronic beats . It received positive reviews , with many critics noting it as a standout from the EP . It peaked at number twenty-one on the US Billboard Hot 100 , and was a top ten hit on the US Hot R & B / Hip-Hop Songs chart . Usher performed the song on The Early Show and on his OMG Tour . = = Background and composition = = The song was leaked","I 'm Not Your Hero = "" I 'm Not Your Hero "" is a song written , recorded and performed by Canadian duo Tegan and Sara and produced by Greg Kurstin for the album Heartthrob ( 2013 ) . Recorded at Echo Studio and EastWest Studios in California in 2012 , it was released on October 21 of that year as a promotional single . The lyrics for the new wave synthpop ballad were written by Sara Quin , and discuss her teenage life in the 1990s when she was unable to identify with the lives of pop culture figures and had unforeseen thoughts about her later life . Guitar chords , drums , bass guitar , synths and harmonized vocals are also played in the track . "" I 'm Not Your Hero "" was mostly well-received by critics for its songwriting and composition . The 



####################################################################################################
-- -- METHOD: elbow


,id_tpc,keys,top doc 1,top doc 2,top doc 3,top doc 4,top doc 5
0,0,"[credits, composition, allmusic, listing, charts, piano, drums, beats, liner, label, recording, download, ballad, songwriter, vocals]","Dancehall Queen ( Robyn song ) = "" Dancehall Queen "" is a song by Swedish recording artist Robyn , taken from her fifth studio album , Body Talk Pt . 1 ( 2010 ) . The song was written by Klas Åhlund , who produced it with disc jockey Diplo . The initial writing and production of the song arose from a discussion by Robyn , Diplo and Åhlund about Ace of Base . The song features a dancehall and reggae-infused sound with 1980s synths and subwoofers . It was released as a promotional single before the album was launched in April 2010 . The song received mixed to positive reviews from critics . Some praised its hook and chorus , while others dismissed its overall sound . "" Dancehall Queen "" charted at number fifty-six on the Sverigetopplistan chart and was listed there for only a week . The accompanying music video , released in November 2010 , was directed by Diplo , Red Foxx and Pomp & Clout . It resembles a karaoke tape with sing-along lyrics on the bottom of the screen . Robyn herself do","Harder to Breathe = "" Harder to Breathe "" is a song by American band Maroon 5 . The song was written by frontman Adam Levine for the band 's debut album , Songs About Jane ( 2002 ) . The song expresses tension , having been written quickly under trying circumstances . It tells the story about a former relationship Levine was involved in . "" Harder to Breathe "" was met with positive reception by music critics , who praised the track 's sound . It was released in 2002 as the lead single of Songs About Jane . The song peaked at number six on Airplay Monitor . It also appeared on the Hot Modern Rock Tracks chart at number 31 and the Billboard Hot 100 at number 18 . Internationally , the single charted at # 13 in the United Kingdom . "" Harder to Breathe "" also appeared in the Netherlands , Sweden , Australia , and New Zealand charts , respectively . The song also appears in Maroon 5 's 2004 EP 1.22.03.Acoustic in an acoustic version and the live album Live – Friday the 13th ( 2005 ) . "" Har","Running ( No Doubt song ) = "" Running "" is a song written by Tony Kanal and Gwen Stefani for No Doubt 's fifth studio album Rock Steady ( 2001 ) and was released worldwide as the album 's fourth and final single on July 1 , 2003 . The song was also used on the last episode of the TV series Sabrina , the Teenage Witch in 2003 . The song received mixed reviews from music critics and was compared to Depeche Mode-style songs . The song only charted on the official charts of the United States , where it became the band 's lowest charting single , and in Germany , where it had longest charting period . The song was accompanied by a music video which was directed by Chris Hafner , which featured many old and new pictures as well as clippings of the band members . = = Background = = The song was written by Stefani and bassist Tony Kanal in Kanal 's living room . They used an old Yamaha keyboard that Kanal 's father had purchased for him when he was in eighth grade and developed the song 's har","La Llamada = "" La Llamada "" ( English : "" The Phone Call "" ) is a song recorded by American recording artist Selena for her first live album Live ! ( 1993 ) . It was composed by Selena y Los Dinos backup singer Pete Astudillo and Selena 's brother and principal record producer A.B. Quintanilla III . The song was produced by Quintanilla III and Argentine music producer Bebu Silvetti . "" La Llamada "" was released as the second single from Live ! . It peaked at number 5 on the US Hot Latin Tracks on the week ending 23 October 1993 . On the week ending 9 April 2011 , "" La Llamada "" entered the Regional Mexican Digital Songs chart . The lyrical content of the song describes a girlfriend 's feelings after witnessing her boyfriend kissing another girl , telling h

## BERTopic

In [63]:
thetas, betas, vocab_w2id, bow, keys = load_model_info(bert_path)

In [44]:
max_thetas = np.max(thetas)
min_thetas = np.min(thetas)
print(f"Max/min thetas: {max_thetas} / {min_thetas}")
thr = (0.5,0.9)

Max/min thetas: 1.0 / 0.0


In [57]:
top_docs_per_topic = []

for doc_distr in thetas.T:
    sorted_docs_indices = np.argsort(doc_distr)[::-1]
    top = sorted_docs_indices[:5].tolist()
    top_docs_per_topic.append(top)
print(top_docs_per_topic[0])
print(thetas[top_docs_per_topic[0],0])
thetas_0 = thetas[:,0]
print(thetas_0[thetas_0 != 0])

[0, 14289, 14288, 14287, 14286]
[0. 0. 0. 0. 0.]
[]


In [64]:
max_thetas = np.max(thetas)
min_thetas = np.min(thetas)
print(f"Max/min thetas: {max_thetas} / {min_thetas}")
thr = (0.5,0.9)

top_docs_per_topic = []

for doc_distr in thetas.T:
    sorted_docs_indices = np.argsort(doc_distr)[::-1]
    top = sorted_docs_indices[:5].tolist()
    top_docs_per_topic.append(top)
print(top_docs_per_topic[0])
print(thetas[top_docs_per_topic[0],0])
thetas_0 = thetas[:,0]
print(thetas_0[thetas_0 != 0])

Max/min thetas: 1.0 / 0.0
[2048, 2057, 2159, 11874, 11837]
[1. 1. 1. 1. 1.]
[1.89075499e-01 1.30882240e-01 6.09278515e-02 6.67913372e-01
 3.38133812e-01 4.85862890e-01 3.30649412e-01 3.85559416e-02
 5.02076638e-01 6.12529423e-02 5.75834876e-02 2.17756821e-01
 1.30344060e-01 6.45948962e-02 2.25830817e-02 4.87116550e-01
 9.16618312e-02 2.49411818e-01 5.17975652e-02 5.60427221e-02
 2.31680329e-02 2.38416499e-01 1.00000000e+00 1.46257810e-01
 7.49744996e-02 1.86752557e-02 4.78921902e-02 2.65531704e-02
 1.12313736e-01 2.32354108e-02 3.26287881e-01 6.54536592e-01
 1.00000000e+00 6.92194924e-02 3.10522592e-02 3.67833967e-02
 1.07317387e-01 3.31774242e-01 2.26930923e-02 1.68831372e-02
 3.27395165e-01 1.98701724e-01 2.78372860e-02 3.41747059e-02
 3.81175311e-02 4.17466005e-01 1.73884639e-01 2.26930923e-02
 1.91190574e-01 5.20239332e-02 2.57535871e-01 4.12218490e-02
 4.78603608e-01 2.77853140e-01 1.00000000e+00 1.97979795e-01
 1.60302371e-01 1.00000000e+00 1.76875120e-01 5.26556459e-02
 2.188841

In [69]:
thetas_7 = thetas[:,7]
print(thetas_7[thetas_7 != 0])

[]


In [70]:
methods = ['thetas', 'thetas_thr']#'sall', 'spart', 's3'

top_docs = {
    method: doc_selector.get_top_docs(method=method, thetas=thetas, bow=bow, betas=betas, corpus=corpus, vocab_w2id=vocab_w2id,thr=thr, top_words = None, ntop=n_top_docs, model_path=bert_path)
    for method in methods
}

In [71]:
for method in top_docs.keys():
    print("##" * 50)
    print(f"-- -- METHOD: {method}")
    df_method = create_top_docs_thetas_dataframe(method, top_docs, df, keys, thetas)
    display(df_method.iloc[0:9])
    (path_save_tops / bert_path.name).mkdir(exist_ok=True)
    this_path_save = path_save_tops / mallet_path.name / f"{method}_top_docs.xlsx"
    df_method.to_excel(this_path_save, index=False)
    print("\n")

####################################################################################################
-- -- METHOD: thetas


,id_tpc,keys,top doc 1,top doc 2,top doc 3,top doc 4,top doc 5
0,0,"[album, song, madonna, chart, video, music, band, songs, harrison, carey, track, pop, recording, vocals, lyrics]",1.00000,1.000000,1.000000,1.000000,1.000000
1,1,"[ship, guns, ships, tons, torpedo, knots, inch, cruiser, fleet, gun, deck, admiral, cruisers, turrets, german]",1.00000,1.000000,1.000000,1.000000,1.000000
2,2,"[episode, mulder, scully, doctor, episodes, trek, series, enterprise, character, viewers, files, amy, television, fringe, scene]",0.49536,0.492988,0.492387,0.487154,0.487154
3,3,"[game, player, gameplay, games, players, playstation, nintendo, released, graphics, characters, soundtrack, xbox, mode, version, manga]",1.00000,1.000000,0.504335,0.296402,0.233116
4,4,"[innings, team, runs, match, league, nba, baseball, batting, season, career, scored, wickets, championship, cricket, basketball]",1.00000,1.000000,0.886863,0.530042,0.519001
5,5,"[highway, route, road, freeway, interchange, intersection, terminus, north, east, continues, lane, state, avenue, passes, crosses]",1.00000,1.000000,1.000000,1.000000,1.000000
6,6,"[tropical, storm, hurricane, winds, depression, cyclone, mph, rainfall, damage, landfall, utc, wind, flooding, weakened, intensity]",1.00000,1.000000,1.000000,1.000000,1.000000
7,7,"[film, films, disney, movie, animated, role, comedy, simpsons, starred, cast, character, production, script, actor, director]",0.00000,0.000000,0.000000,0.000000,0.000000
8,8,"[species, shark, genus, sharks, cap, fruit, stem, fungus, spores, brown, nest, females, habitat, eggs, fin]",1.00000,1.000000,1.000000,1.000000,1.000000




####################################################################################################
-- -- METHOD: thetas_thr


,id_tpc,keys,top doc 1,top doc 2,top doc 3,top doc 4,top doc 5
0,0,"[album, song, madonna, chart, video, music, band, songs, harrison, carey, track, pop, recording, vocals, lyrics]",0.667913,0.666272,0.654537,0.596030,0.509474
1,1,"[ship, guns, ships, tons, torpedo, knots, inch, cruiser, fleet, gun, deck, admiral, cruisers, turrets, german]",0.899718,0.899627,0.899611,0.899185,0.899133
2,2,"[episode, mulder, scully, doctor, episodes, trek, series, enterprise, character, viewers, files, amy, television, fringe, scene]",0.000000,0.000000,0.000000,0.000000,0.000000
3,3,"[game, player, gameplay, games, players, playstation, nintendo, released, graphics, characters, soundtrack, xbox, mode, version, manga]",0.504335,0.000000,0.000000,0.000000,0.000000
4,4,"[innings, team, runs, match, league, nba, baseball, batting, season, career, scored, wickets, championship, cricket, basketball]",0.886863,0.530042,0.519001,0.515383,0.505194
5,5,"[highway, route, road, freeway, interchange, intersection, terminus, north, east, continues, lane, state, avenue, passes, crosses]",0.899802,0.896187,0.893901,0.892122,0.887142
6,6,"[tropical, storm, hurricane, winds, depression, cyclone, mph, rainfall, damage, landfall, utc, wind, flooding, weakened, intensity]",0.899205,0.898431,0.892474,0.889392,0.889129
7,7,"[film, films, disney, movie, animated, role, comedy, simpsons, starred, cast, character, production, script, actor, director]",0.000000,0.000000,0.000000,0.000000,0.000000
8,8,"[species, shark, genus, sharks, cap, fruit, stem, fungus, spores, brown, nest, females, habitat, eggs, fin]",0.719091,0.674786,0.672336,0.669706,0.664072


In [50]:
for method in top_docs.keys():
    print("##" * 50)
    print(f"-- -- METHOD: {method}")
    df_method = create_top_docs_dataframe(method, top_docs, df, keys, thetas)
    display(df_method.iloc[0:9])
    (path_save_tops / ctm_path.name).mkdir(exist_ok=True)
    this_path_save = path_save_tops / ctm_path.name / f"{method}_top_docs.xlsx"
    df_method.to_excel(this_path_save, index=False)
    print("\n")

####################################################################################################
-- -- METHOD: thetas


,id_tpc,keys,top doc 1,top doc 2,top doc 3,top doc 4,top doc 5
0,0,"[episode, mulder, doctor, scully, viewers, episodes, series, television, character, season, watched, dwight, broadcast, trek, jack]","Cicely Mary Barker = Cicely Mary Barker ( 28 June 1895 – 16 February 1973 ) was an English illustrator best known for a series of fantasy illustrations depicting fairies and flowers . Barker 's art education began in girlhood with correspondence courses and instruction at the Croydon School of Art . Her earliest professional work included greeting cards and juvenile magazine illustrations , and her first book , Flower Fairies of the Spring , was published in 1923 . Similar books were published in the following decades . Barker was a devout Anglican , and donated her artworks to Christian fundraisers and missionary organizations . She produced a few Christian-themed books such as The Children ’ s Book of Hymns and , in collaboration with her sister Dorothy , He Leadeth Me . She designed a stained glass window for St. Edmund 's Church , Pitlake , and her painting of the Christ Child , The Darling of the World Has Come , was purchased by Queen Mary . Barker was equally proficient in water","Field hockey pitch = A hockey pitch is the playing surface for the game of field hockey . Historically , the game was played on natural turf ( grass ) but nowadays is predominantly played on an artificial turf . The transition onto artificial pitches came during the 1970s and was made mandatory for major competitions in 1976 . All the lines , markings and goal specifications are outlined by the International Hockey Federation in "" The Rules of Hockey "" . All line markings on the pitch form part of the area which they define . For example , a ball on the side line is still in the field of play ; a ball on the line of the penalty circle is in the penalty circle ; a foul committed over the 23-metre ( 25-yard ) line has occurred in the 23-metre area . A ball must completely cross a boundary line to be out of play , and a ball must wholly cross the goal line before a goal is scored . Due to the original formulation of the rules in England , the standard dimensions of a hockey pitch were ori","Transport in the Soviet Union = Transport in the Union of Soviet Socialist Republics ( USSR ) was an important part of the nation 's economy . The economic centralisation of the late 1920s and 1930s led to the development of infrastructure at a massive scale and rapid pace . Before the Soviet Union 's collapse in 1991 , there were a wide variety of modes of transport by land , water and air . However , because of government policies before , during and after the Era of Stagnation , investments in transport were low . By the late 1970s and early 1980s Soviet economists were calling for the construction of more roads to alleviate some of the strain from the railways and to improve the state budget . The Civil aviation industry , represented by Aeroflot , was the largest in the world , but inefficiencies plagued it until the USSR 's collapse . The road network remained underdeveloped , and dirt roads were common outside major cities . At the same time , the attendance of the few roads the","Amy Wyatt = Amy Marie Violet Wyatt is a fictional character from the British soap opera Emmerdale , played by Chelsea Halfpenny . She debuted on-screen during the episode airing on 8 October 2010 . Since her inception Amy has been central to main storylines and has been played as the serial 's rebellious teenager . Halfpenny has said she enjoys playing the character 's mischievous side . Amy 's unstable upbringing in children 's homes and foster care has impacted on her persona . She is a troubled character and protects herself by creating a hard exterior . Amy 's biggest fear is embarrassment and she aspires to lead a normal family life with her foster parents Val and Eric Pollard . As her storylines have developed she has grown close to the pair , yet rebelled at various sta



####################################################################################################
-- -- METHOD: thetas_thr


,id_tpc,keys,top doc 1,top doc 2,top doc 3,top doc 4,top doc 5
0,0,"[episode, mulder, doctor, scully, viewers, episodes, series, television, character, season, watched, dwight, broadcast, trek, jack]","Cicely Mary Barker = Cicely Mary Barker ( 28 June 1895 – 16 February 1973 ) was an English illustrator best known for a series of fantasy illustrations depicting fairies and flowers . Barker 's art education began in girlhood with correspondence courses and instruction at the Croydon School of Art . Her earliest professional work included greeting cards and juvenile magazine illustrations , and her first book , Flower Fairies of the Spring , was published in 1923 . Similar books were published in the following decades . Barker was a devout Anglican , and donated her artworks to Christian fundraisers and missionary organizations . She produced a few Christian-themed books such as The Children ’ s Book of Hymns and , in collaboration with her sister Dorothy , He Leadeth Me . She designed a stained glass window for St. Edmund 's Church , Pitlake , and her painting of the Christ Child , The Darling of the World Has Come , was purchased by Queen Mary . Barker was equally proficient in water","Field hockey pitch = A hockey pitch is the playing surface for the game of field hockey . Historically , the game was played on natural turf ( grass ) but nowadays is predominantly played on an artificial turf . The transition onto artificial pitches came during the 1970s and was made mandatory for major competitions in 1976 . All the lines , markings and goal specifications are outlined by the International Hockey Federation in "" The Rules of Hockey "" . All line markings on the pitch form part of the area which they define . For example , a ball on the side line is still in the field of play ; a ball on the line of the penalty circle is in the penalty circle ; a foul committed over the 23-metre ( 25-yard ) line has occurred in the 23-metre area . A ball must completely cross a boundary line to be out of play , and a ball must wholly cross the goal line before a goal is scored . Due to the original formulation of the rules in England , the standard dimensions of a hockey pitch were ori","Transport in the Soviet Union = Transport in the Union of Soviet Socialist Republics ( USSR ) was an important part of the nation 's economy . The economic centralisation of the late 1920s and 1930s led to the development of infrastructure at a massive scale and rapid pace . Before the Soviet Union 's collapse in 1991 , there were a wide variety of modes of transport by land , water and air . However , because of government policies before , during and after the Era of Stagnation , investments in transport were low . By the late 1970s and early 1980s Soviet economists were calling for the construction of more roads to alleviate some of the strain from the railways and to improve the state budget . The Civil aviation industry , represented by Aeroflot , was the largest in the world , but inefficiencies plagued it until the USSR 's collapse . The road network remained underdeveloped , and dirt roads were common outside major cities . At the same time , the attendance of the few roads the","Amy Wyatt = Amy Marie Violet Wyatt is a fictional character from the British soap opera Emmerdale , played by Chelsea Halfpenny . She debuted on-screen during the episode airing on 8 October 2010 . Since her inception Amy has been central to main storylines and has been played as the serial 's rebellious teenager . Halfpenny has said she enjoys playing the character 's mischievous side . Amy 's unstable upbringing in children 's homes and foster care has impacted on her persona . She is a troubled character and protects herself by creating a hard exterior . Amy 's biggest fear is embarrassment and she aspires to lead a normal family life with her foster parents Val and Eric Pollard . As her storylines have developed she has grown close to the pair , yet rebelled at various sta